In [ ]:
import numpy as np
import cv2
import time
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
import torch
from pygame import mixer
from datetime import datetime
import requests


ip = "http://192.168.76.115:8000/api/v1/notifications"
method = "POST"


class ObjectDetection:
    def __init__(self, source):
        """Initializes an ObjectDetection instance with a video source (webcam or IP camera URL)."""
        self.source = source

        # Model information
        # Load YOLOv8n model weights
        self.model = YOLO("runs/detect/train/weights/best.pt")

        # Visual information
        self.annotator = None
        self.start_time = 0
        self.end_time = 0

        # Device information
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def predict(self, im0):
        """Run prediction using a YOLO model for the input image im0."""
        results = self.model(im0)
        return results

    def alert(self, class_ids):
        """Send an email alert if a person is detected in the frame."""
        for i in class_ids:
            if i == 3:
                # Send email alert
                print("Garbage thrower!")

                return True

    def display_fps(self, im0):
        """Displays the FPS on an image im0 by calculating and overlaying as white text on a black rectangle."""
        self.end_time = time.time()
        fps = 1 / np.round(self.end_time - self.start_time, 2)
        text = f"FPS: {int(fps)}"
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
        gap = 10
        cv2.rectangle(
            im0,
            (20 - gap, 70 - text_size[1] - gap),
            (20 + text_size[0] + gap, 70 + gap),
            (255, 255, 255),
            -1,
        )
        cv2.putText(im0, text, (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)

    def plot_bboxes(self, results, im0):
        """Plots bounding boxes on an image given detection results; returns annotated image and class IDs."""
        class_ids = []
        # Assuming Annotator class exists for visualization
        im_send = im0.copy()
        self.annotator = Annotator(im_send, 3, results[0].names)
        boxes = results[0].boxes.xyxy.cpu()
        clss = results[0].boxes.cls.cpu().tolist()  # class IDs
        names = results[0].names
        print(names)

        # each box for each class in the image
        for box, cls in zip(boxes, clss):
            class_ids.append(cls)
            self.annotator.box_label(
                box, label=names[int(cls)], color=colors(int(cls), True)
            )  # Assuming colors function exists

        print(class_ids)
        return im_send, class_ids

    def __call__(self):
        """Executes object detection on video frames from a specified camera index, plotting bounding boxes and returning modified frames."""
        cap = cv2.VideoCapture(self.source)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        frame_count = 0
        persisting_frames = 0  # Number of frames that give the same alert
        while True:
            self.start_time = time.time()
            ret, im0 = cap.read()  # im0 is the individual frame.
            assert ret
            results = self.predict(im0)
            im_send, class_ids = self.plot_bboxes(results, im0)

            self.display_fps(im_send)

            cv2.imshow("Perpetrators Detector", im_send)
            mixer.init()
            now = datetime.now()

            if self.alert(class_ids):
                persisting_frames += 1
                if persisting_frames % 10 == 0 and persisting_frames > 0:
                    file_name = now.strftime("%H%M%S") + ".jpg"  # Add ".jpg" extension
                    cv2.imwrite(file_name, im0)
                    
                    data = {
                        "message": "This is a test message",
                        "address": "123 Main Street",
                        "community": "My Community",
                    }
                    with open(file_name, "rb") as f:
                        files = {"file": f}
                        response = requests.request(method, ip, data=data, files=files)
                    if response.status_code == 200:
                        print("good")
                    else:
                        print(response.status_code)

                    
                    print(f"Frame saved as: {file_name}")
                    # mixer.Sound("alert.wav").play()

            print(class_ids)
            if cv2.waitKey(5) & 0xFF == ord("q"):
                break
        cap.release()
        cv2.destroyAllWindows()


# Replace with your IP webcam URL if needed
detector = ObjectDetection(0)

# Call the object detection functionality
detector()

# Check for object detection and send email